In [1]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json


# This secure connect bundle is autogenerated when you download your SCB, 
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'secure-connect-ksandra.zip'
}

# This token JSON file is autogenerated when you download your token, 
# if yours is different update the file name below
with open("ksandra-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

4.0.11-5f20150d0dcb


In [3]:
session.set_keyspace('ksandra')
session.execute("""
CREATE TABLE IF NOT EXISTS clientes (
    cliente_id UUID PRIMARY KEY,
    nombre TEXT,
    direccion TEXT,
    telefono TEXT
);
""")
session.execute("""
CREATE TABLE pedidos_por_cliente (
  cliente_id UUID,
  pedido_id UUID,
  fecha DATE,
  estado TEXT,
  PRIMARY KEY (cliente_id, pedido_id)
);
""")
session.execute("""
CREATE TABLE pedidos_por_fecha (
  fecha DATE,
  pedido_id UUID,
  cliente_id UUID,
  estado TEXT,
  PRIMARY KEY (fecha, pedido_id)
);
""")
session.execute("""
CREATE TABLE productos_por_pedido (
  pedido_id UUID,
  producto_id UUID,
  nombre TEXT,
  cantidad INT,
  precio DECIMAL,
  PRIMARY KEY (pedido_id, producto_id)
);
""")


In [4]:
import uuid
from datetime import date

cliente_id_1 = uuid.uuid4()
cliente_id_2 = uuid.uuid4()

session.execute("""
INSERT INTO clientes (cliente_id, nombre, direccion, telefono)
VALUES (%s, %s, %s, %s)
""", (cliente_id_1, "Ana García", "Calle Falsa 123", "600123456"))

session.execute("""
INSERT INTO clientes (cliente_id, nombre, direccion, telefono)
VALUES (%s, %s, %s, %s)
""", (cliente_id_2, "Luis Pérez", "Av. Principal 456", "699654321"))

pedido_id_1 = uuid.uuid4()
pedido_id_2 = uuid.uuid4()

fecha_1 = date(2024, 3, 1)
fecha_2 = date(2024, 3, 2)

# Pedidos por cliente
session.execute("""
INSERT INTO pedidos_por_cliente (cliente_id, pedido_id, fecha, estado)
VALUES (%s, %s, %s, %s)
""", (cliente_id_1, pedido_id_1, fecha_1, "entregado"))

session.execute("""
INSERT INTO pedidos_por_cliente (cliente_id, pedido_id, fecha, estado)
VALUES (%s, %s, %s, %s)
""", (cliente_id_2, pedido_id_2, fecha_2, "pendiente"))

# Pedidos por fecha
session.execute("""
INSERT INTO pedidos_por_fecha (fecha, pedido_id, cliente_id, estado)
VALUES (%s, %s, %s, %s)
""", (fecha_1, pedido_id_1, cliente_id_1, "entregado"))

session.execute("""
INSERT INTO pedidos_por_fecha (fecha, pedido_id, cliente_id, estado)
VALUES (%s, %s, %s, %s)
""", (fecha_2, pedido_id_2, cliente_id_2, "pendiente"))

producto_id_1 = uuid.uuid4()
producto_id_2 = uuid.uuid4()

session.execute("""
INSERT INTO productos_por_pedido (pedido_id, producto_id, nombre, cantidad, precio)
VALUES (%s, %s, %s, %s, %s)
""", (pedido_id_1, producto_id_1, "Ratón inalámbrico", 1, 19.99))

session.execute("""
INSERT INTO productos_por_pedido (pedido_id, producto_id, nombre, cantidad, precio)
VALUES (%s, %s, %s, %s, %s)
""", (pedido_id_1, producto_id_2, "Teclado mecánico", 1, 49.99))


In [7]:
rows = session.execute("""
SELECT * FROM clientes WHERE cliente_id = %s
""", (cliente_id_1,))
for row in rows:
    print(row)

rows = session.execute("""
SELECT * FROM pedidos_por_cliente WHERE cliente_id = %s
""", (cliente_id_1,))
for row in rows:
    print(row)
rows = session.execute("""
SELECT * FROM productos_por_pedido WHERE pedido_id = %s
""", (pedido_id_1,))
for row in rows:
    print(row)



Row(cliente_id=UUID('30f9745b-637b-4066-b5fd-cbbf5033676a'), direccion='Calle Falsa 123', nombre='Ana García', telefono='600123456')
Row(cliente_id=UUID('30f9745b-637b-4066-b5fd-cbbf5033676a'), pedido_id=UUID('e2574ada-7ff5-4184-81b4-08df6a1c97a6'), estado='entregado', fecha=Date(19783))
Row(pedido_id=UUID('e2574ada-7ff5-4184-81b4-08df6a1c97a6'), producto_id=UUID('ad861f45-b32f-431d-b8a2-94c9e8ac3a59'), cantidad=1, nombre='Ratón inalámbrico', precio=Decimal('19.99'))
Row(pedido_id=UUID('e2574ada-7ff5-4184-81b4-08df6a1c97a6'), producto_id=UUID('bf717f69-239b-4674-bbef-38ab93c36218'), cantidad=1, nombre='Teclado mecánico', precio=Decimal('49.99'))
